<html><head><meta content="text/html; charset=UTF-8" http-equiv="content-type"><style type="text/css">ol</style></head><body class="c5"><p class="c0 c4"><span class="c3"></span></p><p class="c2 title" id="h.rrbabt268i6e"><h1>CaImAn&rsquo;s Demo pipeline</h1></p><p class="c0"><span class="c3">This notebook will help to demonstrate the process of CaImAn and how it uses different functions to denoise, deconvolve and demix neurons from a Calcium Imaging Video. </span></p>
<p><img src="docs/img/quickintro.png" /></p>
<p class="c0"><span class="c3">More information can be found in CaImAn&rsquo;s documentation. </span></p>
</html>



In [ ]:
from IPython.display import YouTubeVideo as yt
from __future__ import division
from __future__ import print_function
from builtins import zip
from builtins import str
from builtins import map
from builtins import range
from past.utils import old_div
import cv2
try:
    cv2.setNumThreads(1)
except:
    print('Open CV is naturally single threaded')

try:
    if __IPYTHON__:
        print((1))
        # this is used for debugging purposes only. allows to reload classes
        # when changed
        get_ipython().magic('load_ext autoreload')
        get_ipython().magic('autoreload 2')
except NameError:
    print('Not IPYTHON')
    pass

import caiman as cm
import numpy as np
import os
import glob
import time
import pylab as pl
import psutil
import sys
from ipyparallel import Client
from skimage.external.tifffile import TiffFile
import scipy

from caiman.motion_correction import tile_and_correct, motion_correction_piecewise
from caiman.source_extraction.cnmf import cnmf as cnmf
from caiman.motion_correction import MotionCorrect
from caiman.components_evaluation import evaluate_components 
from caiman.utils.visualization import plot_contours, view_patches_bar
from caiman.base.rois import extract_binary_masks_blob
from caiman.utils.utils import download_demo
from caiman.utils.visualization import plot_contours,view_patches_bar,nb_plot_contour,nb_view_patches
import bokeh.plotting as bpl
from IPython.lib.display import YouTubeVideo as yt
bpl.output_notebook()

In [ ]:
params_movie = {'fname': ['Sue_2x_3000_40_-46.tif'],
                'niter_rig': 1,
                'max_shifts': (6, 6),  # maximum allow rigid shift
                'splits_rig': 56,  # for parallelization split the movies in  num_splits chuncks across time
                # if none all the splits are processed and the movie is saved
                'num_splits_to_process_rig': None,
                # intervals at which patches are laid out for motion correction
                'strides': (48, 48),
                # overlap between pathes (size of patch strides+overlaps)
                'overlaps': (24, 24),
                'splits_els': 56,  # for parallelization split the movies in  num_splits chuncks across time
                # if none all the splits are processed and the movie is saved
                'num_splits_to_process_els': [28, None],
                'upsample_factor_grid': 4,  # upsample factor to avoid smearing when merging patches
                # maximum deviation allowed for patch with respect to rigid
                # shift
                'max_deviation_rigid': 3,
                'p': 1,  # order of the autoregressive system
                'merge_thresh': 0.8,  # merging threshold, max correlation allowed
                'rf': 15,  # half-size of the patches in pixels. rf=25, patches are 50x50
                'stride_cnmf': 6,  # amounpl.it of overlap between the patches in pixels
                'K': 4,  # number of components per patch
                # if dendritic. In this case you need to set init_method to
                # sparse_nmf
                'is_dendrites': False,
                'init_method': 'greedy_roi',
                'gSig': [4, 4],  # expected half size of neurons
                'alpha_snmf': None,  # this controls sparsity
                'final_frate': 30
                }

In [ ]:
#%% parameters from dictionary
fname = params_movie['fname']
niter_rig = params_movie['niter_rig']
# maximum allow rigid shift
max_shifts = params_movie['max_shifts']  
# for parallelization split the movies in  num_splits chuncks across time
splits_rig = params_movie['splits_rig']  
# if none all the splits are processed and the movie is saved
num_splits_to_process_rig = params_movie['num_splits_to_process_rig']
# intervals at which patches are laid out for motion correction
strides = params_movie['strides']
# overlap between pathes (size of patch strides+overlaps)
overlaps = params_movie['overlaps']
# for parallelization split the movies in  num_splits chuncks across time
splits_els = params_movie['splits_els'] 
# if none all the splits are processed and the movie is saved
num_splits_to_process_els = params_movie['num_splits_to_process_els']
# upsample factor to avoid smearing when merging patches
upsample_factor_grid = params_movie['upsample_factor_grid'] 
# maximum deviation allowed for patch with respect to rigid
# shift
max_deviation_rigid = params_movie['max_deviation_rigid']
#%% some parameter settings
# order of the autoregressive fit to calcium imaging in general one (slow gcamps) or two (fast gcamps fast scanning)
p = params_movie['p']  
# merging threshold, max correlation allowed
merge_thresh= params_movie['merge_thresh'] 
# half-size of the patches in pixels. rf=25, patches are 50x50
rf = params_movie['rf']  
# amounpl.it of overlap between the patches in pixels
stride_cnmf = params_movie['stride_cnmf'] 
 # number of components per patch
K =  params_movie['K'] 
# if dendritic. In this case you need to set init_method to sparse_nmf
is_dendrites = params_movie['is_dendrites']
# iinit method can be greedy_roi for round shapes or sparse_nmf for denritic data
init_method = params_movie['init_method']
# expected half size of neurons
gSig = params_movie['gSig']  
# this controls sparsity
alpha_snmf = params_movie['alpha_snmf']  
#frame rate of movie (even considering eventual downsampling)
final_frate = params_movie['final_frate']
#%% Extract spatial and temporal components on patches

In [ ]:
# %% download movie if not there                                                                                                                                                                                
if fname[0] in ['Sue_2x_3000_40_-46.tif','demoMovieJ.tif']:
    download_demo(fname[0])
    fname = [os.path.join('example_movies',fname[0])]
m_orig = cm.load_movie_chain(fname[:1])
yt('mLvNmPs9onE')
print("Data from Sue Ann Koay, Princeton Neuroscience Institute, Tank Lab.")

In [ ]:
#%% load movie (in memory!)
m_orig = cm.load_movie_chain(fname)
#%% play movie
downsample_ratio = .2
offset_mov = -np.min(m_orig[:100])
m_orig.resize(1, 1, downsample_ratio).play(
    gain=10, offset = offset_mov, fr=30, magnification=2)

<em>some of the pixels in this movie are negative, we then need to make them positive </em>

<h2> Motion correction is performed in parallel on chunks taken across times. </h2>
<p> Create temporal chunks of the movie for  parallel processing on all cores </p>
<p> ipyparallel is used to create a cluster that handles the parallelization eithr on the PC ( see : https://ipyparallel.readthedocs.io/en/latest/intro.html) or on clusters interfacing with slurm ( see : https://slurm.schedmd.com/quickstart.html ) </p>
<p><img src="docs/img/cordermmap.png" /> </p>

In [ ]:
#%% start the cluster
c, dview, n_processes = cm.cluster.setup_cluster(
    backend='local', n_processes=None, single_thread=False)

<h1> Rigid motion correction</h1>
<p> We are now left with a really shacky video. In order to correct the image movement we are using a simple rigid motion correction algorithm. This algorithm first creates a correlation image over time using frames from the video. It then tries to match each frame to this template. In addition the template will get updated during the matching process. Making it more precise and so does the template matching.  </p>
<img src="docs/img/rigidcorrection.png" />
more info : <em> http://opencv.org/about.html </em>


In [ ]:
# movie must be mostly positive for this to work
min_mov = cm.load(fname[0], subindices=range(400)).min()

mc = MotionCorrect(fname[0], min_mov,
                   dview=dview, max_shifts=max_shifts, niter_rig=niter_rig,
                   splits_rig=splits_rig, 
                   num_splits_to_process_rig=num_splits_to_process_rig, 
                strides= strides, overlaps= overlaps, splits_els=splits_els,
                num_splits_to_process_els=num_splits_to_process_els, 
                upsample_factor_grid=upsample_factor_grid,
                   max_deviation_rigid=max_deviation_rigid, 
                shifts_opencv = True, nonneg_movie = True)

mc.motion_correct_rigid(save_movie=True)
# load motion corrected movie
m_rig = cm.load(mc.fname_tot_rig)
bord_px_rig = np.ceil(np.max(mc.shifts_rig)).astype(np.int)
#%% visualize templates
pl.figure(figsize = (20,10))
pl.imshow(mc.total_template_rig, cmap = 'gray')
#%% inspect movie
m_rig.resize(1, 1, downsample_ratio).play(
    gain=10, offset = offset_mov*.25, fr=30, magnification=2,bord_px = bord_px_rig)

<h2> Inspect motion correction results</h2>

In [ ]:
#%% plot rigid shifts
pl.close()
pl.figure(figsize = (20,10))
pl.plot(mc.shifts_rig)
pl.legend(['x shifts','y shifts'])
pl.xlabel('frames')
pl.ylabel('pixels')

<h1> Piecewise rigid motion correction </h1>
<p> our video is now way less shaky but since the brain tissue is a compressible object, a non  rigid deformation still exist and in order to do a really nice source separation, needs to be corrected : </p>
<p> Here we introduce a function for a fast Non-Rigid Motion Correction based on template matching.<br/>It is a similar approach than the one presented before but on overlapping spatial patches. The estimated alignments are subsequently up-sampled to create a smooth motion field for each frame that can efficiently approximate non-rigid motion in a piecewise-rigid manner.<br/>It can be run in an online mode resulting in comparable to or even
faster than real time motion registration on streaming data.<p>
<img src="docs\img\pwrigidcorrection.png" />
<p> more info : (Eftychios A. Pnevmatikakis, Andrea Giovannucci, NormCorre) </p>
<em> http://biorxiv.org/content/biorxiv/early/2017/02/14/108514.full.pdf </em>

In [ ]:
#%% motion correct piecewise rigid
mc.motion_correct_pwrigid(save_movie=True, template=mc.total_template_rig,
                          show_template = True)
m_els = cm.load(mc.fname_tot_els)
pl.figure(figsize = (20,10))
pl.imshow(mc.total_template_els, cmap = 'gray')
yt('JzierjDuBzg')

In [ ]:
#%% visualize elastic shifts
pl.close()
pl.figure(figsize = (20,10))
pl.subplot(2, 1, 1)
pl.plot(mc.x_shifts_els)
pl.ylabel('x shifts (pixels)')
pl.subplot(2, 1, 2)
pl.plot(mc.y_shifts_els)
pl.ylabel('y_shifts (pixels)')
pl.xlabel('frames')
#%% compute borders to exclude
bord_px_els = np.ceil(np.maximum(np.max(np.abs(mc.x_shifts_els)),
                                 np.max(np.abs(mc.y_shifts_els)))).astype(np.int)

In [ ]:
# check movie
downsample_ratio = .2
m_els.resize(1, 1, downsample_ratio).play(
    gain=10, offset = 0, fr=30, magnification=2,bord_px = bord_px_els)
# compare with original and rigid corrected movies
downsample_factor = .2
cm.concatenate([m_orig.resize(1, 1, downsample_factor)+offset_mov,
                m_rig.resize(1, 1, downsample_factor), m_els.resize(
    1, 1, downsample_factor)], axis=2).play(fr=60, gain=15, magnification=2, offset=0)
#%% local correlation
pl.figure(figsize = (20,10))
pl.imshow(m_els.local_correlations(eight_neighbours=True, swap_dim=False))

<h1> Assessing Motion correction Quality </h1>
for the raw, rigid corrected and piecewise rigid corrected videos
<p> using smoothness <em> (mean over time) </em> </p>
--------------
<p> using correlation to the template <em> (Pearson Correlation coefficient)</em> </p> 
--------------
see : http://docs.opencv.org/2.4/doc/tutorials/imgproc/histograms/template_matching/template_matching.html ( like a normalized SSD )
<p> optical flow : </p>
--------------
<img src="docs/img/opticalflow.png" />
more info :<em> http://docs.opencv.org/trunk/d7/d8b/tutorial_py_lucas_kanade.html </em>

In [ ]:
#% compute metrics for the results (TAKES TIME!!)
final_size = np.subtract(mc.total_template_els.shape, 2 * bord_px_els)
winsize = 100
swap_dim = False
resize_fact_flow = .2
tmpl, correlations, flows_orig, norms, smoothness = cm.motion_correction.compute_metrics_motion_correction(
    mc.fname_tot_els, final_size[0], final_size[1],
    swap_dim, winsize=winsize, play_flow=False, resize_fact_flow=resize_fact_flow)

tmpl, correlations, flows_orig, norms, smoothness = cm.motion_correction.compute_metrics_motion_correction(
    mc.fname_tot_rig, final_size[0], final_size[1],
    swap_dim, winsize=winsize, play_flow=False, resize_fact_flow=resize_fact_flow)

tmpl, correlations, flows_orig, norms, smoothness = cm.motion_correction.compute_metrics_motion_correction(
    fname[0], final_size[0], final_size[1], swap_dim, winsize=winsize, play_flow=False, resize_fact_flow=resize_fact_flow)

In [ ]:
#%% plot the results of metrics
fls = [mc.fname_tot_els[:-4] + '_metrics.npz', mc.fname_tot_rig[:-4] +
       '_metrics.npz', mc.fname[:-4] + '_metrics.npz']

pl.figure(figsize = (20,10))
for cnt, fl, metr in zip(range(len(fls)),fls,['pw_rigid','rigid','raw']):
    with np.load(fl) as ld:
        print(ld.keys())
        print(fl)
        print(str(np.mean(ld['norms'])) + '+/-' + str(np.std(ld['norms'])) +
              ' ; ' + str(ld['smoothness']) + ' ; ' + str(ld['smoothness_corr']))
        
        pl.subplot(len(fls), 4, 1 + 4 * cnt)
        pl.ylabel(metr)
        try:
            mean_img = np.mean(
            cm.load(fl[:-12] + 'mmap'), 0)[12:-12, 12:-12]
        except:
            try:
                mean_img = np.mean(
                    cm.load(fl[:-12] + '.tif'), 0)[12:-12, 12:-12]
            except:
                mean_img = np.mean(
                    cm.load(fl[:-12] + 'hdf5'), 0)[12:-12, 12:-12]
                    
        lq, hq = np.nanpercentile(mean_img, [.5, 99.5])
        pl.imshow(mean_img, vmin=lq, vmax=hq)
        pl.title('Mean')
        pl.subplot(len(fls), 4, 4 * cnt + 2)
        pl.imshow(ld['img_corr'], vmin=0, vmax=.35)
        pl.title('Corr image')
        pl.subplot(len(fls), 4, 4 * cnt + 3)
        pl.plot(ld['norms'])
        pl.xlabel('frame')
        pl.ylabel('norm opt flow')
        pl.subplot(len(fls), 4, 4 * cnt + 4)
        flows = ld['flows']
        pl.imshow(np.mean(
        np.sqrt(flows[:, :, :, 0]**2 + flows[:, :, :, 1]**2), 0), vmin=0, vmax=0.3)
        pl.colorbar()
        pl.title('Mean optical flow')      

<h1> Memory mapping </h1>

<p> We want the parallel processes to access and our video matrix without having it in memory and duplicating it. we are using the memapping method to fullfill such a goal</p>
<p> learn more : https://en.wikipedia.org/wiki/Memory-mapped_file </p>
<p><img src="docs/img/fordermmap.png" /></p>
<h3> Data is saved on drive so that extracting blocks of movies is efficient.  </h3>
<p>Data can be either quickly read per columns or per rows but not in both directions. So, we save the movie in such a way that it is easy to read across time.</p>


In [ ]:
# MEMORY MAPPING: save each chunk in F format on memory mapped files
if 'max_shifts' not in params_movie:
    fnames = [params_movie['fname']]
    border_to_0 = 0
elif not 'overlaps'in params_movie:
    fnames = [mc.fname_tot_rig]
    border_to_0 = bord_px_rig
    m_els = m_rig
else:
    fnames = [mc.fname_tot_els]
    border_to_0 = bord_px_els
    
idx_xy = None
add_to_movie = -np.nanmin(m_els) + 1  # movie must be positive
# if you need to remove frames from the beginning of each file
remove_init = 0
# downsample movie in time: use .2 or .1 if file is large and you want a quick answer             
downsample_factor = 1 
base_name = fname[0].split('/')[-1][:-4]
name_new = cm.save_memmap_each(fnames, dview=dview, base_name=base_name, resize_fact=(
    1, 1, downsample_factor), remove_init=remove_init, idx_xy=idx_xy,
                               add_to_movie=add_to_movie, border_to_0=border_to_0)
name_new.sort()
print(name_new)

#%% concatenate chunks if needed
if len(name_new) > 1:
    fname_new = cm.save_memmap_join(
        name_new, base_name='Yr', n_chunks=12, dview=dview)
else:
    print('One file only, not saving!')
    fname_new = name_new[0]

In [ ]:
#%% LOAD MEMMAP FILE
Yr, dims, T = cm.load_memmap(fname_new)
d1, d2 = dims
images = np.reshape(Yr.T, [T] + list(dims), order='F')
Y = np.reshape(Yr, dims + (T,), order='F')
m_images = cm.movie(images)
#%%  checks on movies (might take time if large!)
if np.min(images) < 0:
    raise Exception('Movie too negative, add_to_movie should be larger')
if np.sum(np.isnan(images)) > 0:
    raise Exception('Movie contains nan! You did not remove enough borders')
#%% correlation image
Cn = cm.local_correlations(Y)
Cn[np.isnan(Cn)] = 0
pl.imshow(Cn, cmap='gray', vmax=.35)

<h1> CNMF on patches </h1>
<p> Our method builds upon and extend (Pnevmatikakis et al, Neuron, 2016)  <em> http://www.cell.com/neuron/fulltext/S0896-6273(15)01084-3 </em> </p>

<p> <img src="docs/img/cnmf1.png" /> </p>

<p> We run portion of the CNMF algorithm  on patches </p>
<p> We merge the patches with special attention to neurons on the border. </p>
<p> We refine the result by rerunning CNMF on the whole movie </p>

In [ ]:
cnm = cnmf.CNMF(n_processes, k=K, gSig=gSig, merge_thresh=0.8, p=0, 
                dview=dview, Ain=None, rf=rf, stride=stride_cnmf, memory_fact=1,
                method_init=init_method, alpha_snmf=alpha_snmf,
                only_init_patch=True, gnb=1, method_deconvolution='oasis')
cnm = cnm.fit(images)

A_tot = cnm.A
C_tot = cnm.C
YrA_tot = cnm.YrA
b_tot = cnm.b
f_tot = cnm.f
sn_tot = cnm.sn
print(('Number of components:' + str(A_tot.shape[-1])))
pl.figure()
crd = plot_contours(A_tot, Cn, thr=0.9)

<h1>DISCARD LOW QUALITY COMPONENT </h1>
<p> The patch dubdivision creates several spurious components that are not neurons </p>
<p>We select the components according to criteria examining spatial and temporal components</p>
<img src="docs/img/evaluationcomponent.png"/>

<p> Temporal components, for each trace: </p>

<li>  compute the robust mode, corresponding to the baseline value</li>
<li> use the values under the mode to estimate noise variance</li>
<li> compute the probability of having large transients  given the noise distribution estimated </li>
<li> Threshold on this probability s.t. some of the component are discarded because lacking large enough positive transients </li>

<p> Spatial components, for each components: </p>

<li> average the frames in the moveie where the neurons is active (from temporal component), this provides a nice image of the neuron</li>
<li> compare this image with the corresponding spatial component (Person's correlation coefficient)</li>
<li> threshold the correlation coefficient  </li>


In [ ]:
#%% DISCARD LOW QUALITY COMPONENT
from caiman.components_evaluation import estimate_components_quality
final_frate = params_movie['final_frate']
r_values_min = .7  # threshold on space consistency
fitness_min = -40  # threshold on time variability
# threshold on time variability (if nonsparse activity)
fitness_delta_min = -40
Npeaks = 10
traces = C_tot + YrA_tot
idx_components, idx_components_bad = estimate_components_quality(
    traces, Y, A_tot, C_tot, b_tot, f_tot, final_frate=final_frate, 
    Npeaks=Npeaks, r_values_min=r_values_min, fitness_min=fitness_min,
    fitness_delta_min=fitness_delta_min)
print(('Keeping ' + str(len(idx_components)) +
       ' and discarding  ' + str(len(idx_components_bad))))
#%%
pl.figure(figsize=(20,10))
crd = plot_contours(A_tot.tocsc()[:, idx_components], Cn, thr=0.9)


<h1> CNMF full Field of View </h1> 

In [ ]:
#%%
#A_tot = A_tot.tocsc()[:, idx_components]
#C_tot = C_tot[idx_components]
#%% rerun updating the components to refine
cnm = cnmf.CNMF(n_processes, k=A_tot.shape, gSig=gSig, 
                merge_thresh=merge_thresh, p=0, dview=dview, Ain=A_tot, Cin=C_tot,
                f_in=f_tot, rf=None, stride=None, method_deconvolution='oasis')
cnm = cnm.fit(images)
#%%
A, C, b, f, YrA, sn = cnm.A, cnm.C, cnm.b, cnm.f, cnm.YrA, cnm.sn

<h1> Discard low quality components on full field of view</h1>


In [ ]:
#%% again recheck quality of components, stricter criteria
final_frate = params_movie['final_frate']
r_values_min = .75
fitness_min = - 50
fitness_delta_min = - 50
Npeaks = 10
traces = C + YrA
idx_components, idx_components_bad = estimate_components_quality(
    traces, Y, A, C, b, f, final_frate=final_frate,
    Npeaks=Npeaks, r_values_min=r_values_min, fitness_min=fitness_min,
    fitness_delta_min=fitness_delta_min)
print(' ***** ')
print((len(traces)))
print((len(idx_components)))
print(len(idx_components_bad))

In [ ]:
#%% visualize included and excluded components
pl.figure(figsize=(20,10))
pl.subplot(1, 2, 1)
crd = plot_contours(A.tocsc()[:, idx_components], Cn, thr=0.9)
pl.subplot(1, 2, 2)
crd = plot_contours(A.tocsc()[:, idx_components_bad], Cn, thr=0.9)

# kept components

In [ ]:
discard_traces_fluo=nb_view_patches(Yr,A.tocsc()[:,idx_components],
                                    C[idx_components],b,f,dims[0],
                                    dims[1],thr = 0.8,image_neurons=Cn,
                                    denoised_color='red')
print('you will may be need to change the data rate to generate this one: use jupyter notebook --NotebookApp.iopub_data_rate_limit=1.0e10')

# discarded components

In [ ]:
discard_traces_fluo=nb_view_patches(Yr,A.tocsc()[:,idx_components_bad],
                                    C[idx_components_bad],b,f,dims[0],dims[1],
                                    thr = 0.8,image_neurons=Cn, denoised_color='red')

we can see here a display of the components that have been kept throught the cnmf process and the one that have been discarded. 

<h1> closing, saving, and creating denoised version </h1>

In [ ]:
np.savez(os.path.join(os.path.split(fname_new)[0], 
                      os.path.split(fname_new)[1][:-4] + 'results_analysis.npz'),
         Cn=Cn, A=A.todense(), C=C,
         b=b, f=f, YrA=YrA, sn=sn, d1=d1, d2=d2,
         idx_components=idx_components, idx_components_bad=idx_components_bad)

In [ ]:
#%% STOP CLUSTER and clean up log files
cm.stop_server()
log_files = glob.glob('*_LOG_*')
for log_file in log_files:
    os.remove(log_file)

In [ ]:
#%% reconstruct denoised movie without background
denoised = cm.movie(A.dot(C)).reshape(dims+(-1,),order = 'F').transpose([2,0,1])
#%%
denoised.play(gain = 30, offset = 0,fr =100, magnification = 2)
